In [ ]:
# Code adapted from: https://github.com/aladdinpersson/Machine-Learning-Collection/blob/ac5dcd03a40a08a8af7e1a67ade37f28cf88db43/ML/Pytorch/GANs/2.%20DCGAN/train.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as tfms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import os
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
def noise_dims(map_dims):
    # For each convtranspose layer:
    # dim = round(math.ceil((dim + (2*padding) - kernel_size) / stride)) + 1
    # The output will need to be trimmed down to the desired size (This may hurt generalizability)
    noise_shape = []
    for i in range(len(map_dims)):
        shape = map_dims[i]

        shape = ((shape + (2*1) - 4) // 2) + 1  # Layer 5
        shape = ((shape + (2*1) - 4) // 2) + 1  # Layer 4
        shape = ((shape + (2*1) - 4) // 2) + 1  # Layer 3
        shape = ((shape + (2*1) - 4) // 2) + 1  # Layer 2
        shape = ((shape + (2*0) - 4) // 2) + 1  # Layer 1

        # shape += 1  # Trim the generated paths down later
        noise_shape.append(shape)

    noise_shape = tuple(noise_shape)
    return noise_shape

In [ ]:
# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MAP_NAME = "map_64x64"
MAP_DIMS = (64,64)

FEATURES_GEN = 64
FEATURES_DISC = 64
NOISE_DIM = 100
NOISE_SHAPE = noise_dims(MAP_DIMS)
IMG_CHANNELS = 1
LEARNING_RATE = 5e-4
BATCH_SIZE = 64
NUM_EPOCHS = 20

#Specific to WGAN
WEIGHT_CLIP = 0.01 # C param from WGAN paper
CRITIC_ITERATIONS = 5 # how many times the critic loop runs for each generator loop

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d, device='cpu'):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(
                channels_img, features_d, kernel_size=4, stride=2, padding=1, device=device
            ),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding, device=device)
            self._block(features_d, features_d * 2, 4, 2, 1, device=device),
            self._block(features_d * 2, features_d * 4, 4, 2, 1, device=device),
            self._block(features_d * 4, features_d * 8, 4, 2, 1, device=device),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0, device=device)
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding, device):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False, device=device,
            ),
            nn.BatchNorm2d(out_channels, device=device),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g, device='cpu'):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 16, 4, 1, 0, device=device),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1, device=device),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1, device=device),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1, device=device),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1, device=device
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding, device):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False, device=device,
            ),
            nn.BatchNorm2d(out_channels, device=device),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

In [ ]:
# Need to override __init__, __len__, __getitem__
# as per datasets requirement
class PathsDataset(torch.utils.data.Dataset):
    # init the dataset, shape = L x W
    def __init__(self, path, transform=None, shape = (100,100)):
        self.paths = [] # create a list to hold all paths read from file
        for filename in os.listdir(path):
            with open(os.path.join(path, filename), 'r') as f: # open in readonly mode
                self.flat_path = np.loadtxt(f) # load in the flat path from file
                self.path = np.asarray(self.flat_path, dtype=int).reshape(len(self.flat_path)//2,2) #unflatten the path from the file

                self.path_matrix = self.convert_path(shape, self.path)
                
                self.paths.append(self.path_matrix) # add the path to paths list
        self.transform = transform
        print("Done!")

    def convert_path(self, map_dim, path):
        path_mat = np.zeros(map_dim, dtype=float)

        # Make the path continuous
        for i in range(path.shape[0] - 1):
            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            if (x1 < x2):
                x_dir = 1
            else:
                x_dir = -1

            if (y1 < y2):
                y_dir = 1
            else:
                y_dir = -1

            # Determine y from x
            if x2-x1 != 0:
                m = (y2-y1)/(x2-x1)
                while x != x2:
                    y = round(m*(x-x1) + y1)
                    path_mat[y,x] = 1
                    x += x_dir
            else:
                while x != x2:
                    path_mat[y1,x] = 1
                    x += x_dir


            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            # Determine x from y
            if y2-y1 != 0:
                m = (x2-x1)/(y2-y1)
                while y != y2:
                    x = round(m*(y-y1) + x1)
                    path_mat[y,x] = 1
                    y += y_dir
            else:
                while y != y2:
                    path_mat[y,x1] = 1
                    y += y_dir
            
        path_mat[path[path.shape[0]-1,1], path[path.shape[0]-1,0]] = 1     # Include the last point in the path

        return path_mat

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        x = np.float32(self.paths[idx])

        if self.transform:
            x = self.transform(x).cuda()
            

        return x

        #return image, label

In [ ]:
transforms = tfms.Compose(
    [
        tfms.ToTensor(),
        tfms.Normalize(
            [0.5 for _ in range(IMG_CHANNELS)], [0.5 for _ in range(IMG_CHANNELS)]
        ),
    ]
)

In [ ]:
dataset = PathsDataset(path = f"./env/{MAP_NAME}/paths/80000_paths/", shape = MAP_DIMS, transform=transforms)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

gen = Generator(NOISE_DIM, IMG_CHANNELS, FEATURES_GEN, device=device)
critic = Discriminator(IMG_CHANNELS, FEATURES_DISC, device=device)
initialize_weights(gen)
initialize_weights(critic)

In [ ]:
opt_gen = optim.RMSprop(gen.parameters(), lr=LEARNING_RATE)
opt_critic = optim.RMSprop(critic.parameters(), lr=LEARNING_RATE)

In [ ]:
fixed_noise = torch.randn((BATCH_SIZE, NOISE_DIM, NOISE_SHAPE[0], NOISE_SHAPE[1]), device=device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0

In [ ]:
gen.train()
critic.train()

In [ ]:
for epoch in range(NUM_EPOCHS):
    # Target labels not needed! <3 unsupervised
    # for batch_idx, (real, _) in enumerate(dataloader):
    for batch_idx, real in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn((BATCH_SIZE, NOISE_DIM, NOISE_SHAPE[0], NOISE_SHAPE[1]), device=device)
        fake = gen(noise)

        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn((BATCH_SIZE, NOISE_DIM, NOISE_SHAPE[0], NOISE_SHAPE[1]), device=device)
            fake = gen(noise)
            critic_real = critic(real).reshape(-1)
            critic_fake = critic(fake).reshape(-1)
            loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake)) #   want to maximize (according to paper) but 
                                                                            #   optim algorithms are for minimizing so take - 
            critic.zero_grad()
            loss_critic.backward(retain_graph=True) # want to re use the computations for fake for generator
            opt_critic.step()

            for p in critic.parameters():
                p.data.clamp_(-WEIGHT_CLIP, WEIGHT_CLIP)

        ### Training generator: min E(critic(gen_fake))
        output = critic(fake).reshape(-1)
        loss_gen = -torch.mean(output)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()        

        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:BATCH_SIZE], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:BATCH_SIZE], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1

In [ ]:
noise = torch.randn((BATCH_SIZE, NOISE_DIM, NOISE_SHAPE[0], NOISE_SHAPE[1]), device=device)
fake = gen(noise)

In [ ]:
plt.imshow(fake.cpu().detach().numpy()[7][0])